In [9]:
import numpy as np
import pandas as pd
import os
from sklearn.preprocessing import MinMaxScaler
import joblib
from config import LIST_TICKETS

print(f"Ações disponíveis para treino: {LIST_TICKETS}")
# --- Configurações ---
# REMOVEMOS AQUI A LINHA: TICKER_PARA_PROCESSAR = 'VALE3.SA'
CAMINHO_DADOS_BRUTOS = "../data/01_raw"
CAMINHO_DADOS_PROCESSADOS = "../data/03_processed"
CAMINHO_MODELOS = "../models"

# Parâmetros para o pré-processamento
PERCENTUAL_TREINO = 0.8
JANELA_DE_TEMPO = 60 # Usaremos 60 dias de histórico para prever o próximo

# Garante que os diretórios de saída existam
os.makedirs(CAMINHO_DADOS_PROCESSADOS, exist_ok=True)
os.makedirs(CAMINHO_MODELOS, exist_ok=True)

print("Configurações carregadas.")

Ações disponíveis para treino: ['PETR4.SA', 'VALE3.SA', 'ITUB4.SA']
Configurações carregadas.


In [10]:
def processar_e_salvar_dados(ticker):
    print(f"\n----- Iniciando Processamento para {ticker} -----")

    # 1. Carregamento dos dados
    caminho_arquivo = os.path.join(CAMINHO_DADOS_BRUTOS, f"{ticker}.csv")
    try:
        # Importante: A coluna que contém o preço a ser previsto deve ser a ÚNICA selecionada.
        # No seu log anterior, a coluna de fechamento (Close) era o índice 1 (Unnamed: 1).
        df = pd.read_csv(
            caminho_arquivo,
            header=2,
            index_col='Date',
            parse_dates=True
        )
    except FileNotFoundError:
        print(f"ERRO: Arquivo não encontrado em '{caminho_arquivo}'. Pulando.")
        return

    # Tratamento de valores nulos (se houver) e Seleção da coluna
    df.dropna(inplace=True)
    # Selecionamos a coluna 1, que é o fechamento ajustado (Close)
    dados_fechamento = df.iloc[:, 1].values.reshape(-1, 1)

    # 2. Divisão e Normalização
    ponto_divisao = int(len(dados_fechamento) * PERCENTUAL_TREINO)
    dados_treino = dados_fechamento[:ponto_divisao]
    dados_teste = dados_fechamento[ponto_divisao:]

    scaler = MinMaxScaler(feature_range=(0, 1))
    scaler.fit(dados_treino)

    dados_treino_normalizados = scaler.transform(dados_treino)
    dados_teste_normalizados = scaler.transform(dados_teste)

    # CORREÇÃO CRÍTICA: Salva o scaler no local CORRETO para o treinamento
    caminho_scaler = os.path.join(CAMINHO_DADOS_PROCESSADOS, f"{ticker}_scaler.joblib")
    joblib.dump(scaler, caminho_scaler)
    print(f"Scaler salvo com sucesso em: {caminho_scaler}")

    # 3. Criação das Sequências
    def criar_sequencias(dados, janela_de_tempo):
        X, y = [], []
        for i in range(janela_de_tempo, len(dados)):
            X.append(dados[i-janela_de_tempo:i, 0])
            y.append(dados[i, 0])
        return np.array(X), np.array(y)

    X_train, y_train = criar_sequencias(dados_treino_normalizados, JANELA_DE_TEMPO)
    X_test, y_test = criar_sequencias(dados_teste_normalizados, JANELA_DE_TEMPO)

    # Ajusta o formato de X para ser [amostras, janelas, features]
    X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
    X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

    # 4. Salvamento dos Arrays
    np.save(os.path.join(CAMINHO_DADOS_PROCESSADOS, f'{ticker}_X_train.npy'), X_train)
    np.save(os.path.join(CAMINHO_DADOS_PROCESSADOS, f'{ticker}_y_train.npy'), y_train)
    np.save(os.path.join(CAMINHO_DADOS_PROCESSADOS, f'{ticker}_X_test.npy'), X_test)
    np.save(os.path.join(CAMINHO_DADOS_PROCESSADOS, f'{ticker}_y_test.npy'), y_test)

    print(f"Dados processados ({X_train.shape[0]} amostras de treino) foram salvos para {ticker}.")


# --- EXECUÇÃO DO LOOP PRINCIPAL ---
for ticker in LIST_TICKETS:
    processar_e_salvar_dados(ticker)

print("\n\n✅ Processamento de todos os Tickers concluído!")


----- Iniciando Processamento para PETR4.SA -----
Scaler salvo com sucesso em: ../data/03_processed/PETR4.SA_scaler.joblib
Dados processados (5129 amostras de treino) foram salvos para PETR4.SA.

----- Iniciando Processamento para VALE3.SA -----
Scaler salvo com sucesso em: ../data/03_processed/VALE3.SA_scaler.joblib
Dados processados (5129 amostras de treino) foram salvos para VALE3.SA.

----- Iniciando Processamento para ITUB4.SA -----
Scaler salvo com sucesso em: ../data/03_processed/ITUB4.SA_scaler.joblib
Dados processados (4927 amostras de treino) foram salvos para ITUB4.SA.


✅ Processamento de todos os Tickers concluído!
